# IMPORT FUNCFIONS

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import pickle
import os
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import average_precision_score
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from ge import LINE

# READ DATA

In [2]:
def get_edges_dict(traindata):
    if os.path.isfile("edgesdic.pkl"):
        f = open("edgesdic.pkl","rb")
        edges = pickle.load(f)
        f.close()
        return edges
    else:
        edges = dict()
        for edge in range(len(traindata)): 
            edges[(traindata.loc[[edge]]['id_1'].values[0],traindata.loc[[edge]]['id_2'].values[0])] = 1 
            edges[(traindata.loc[[edge]]['id_2'].values[0],traindata.loc[[edge]]['id_1'].values[0])] = 1  
        f = open("edgesdic.pkl","wb")
        pickle.dump(edges,f)
        f.close()
        return edges

def get_negative_edges(traindata,g):
    edges = get_edges_dict(traindata)
    # for missing edges.
    negative_edges = set([])
    maxNodenum = max(max(traindata['id_1']),max(traindata['id_2']))
    #產生與原圖同edge數目的negative edges
    while (len(negative_edges)<len(traindata)):
        node1=random.randint(0, maxNodenum) 
        node2=random.randint(0, maxNodenum) 
        tmp = edges.get((node1,node2),0) #edge不在graph裡標為0
        if tmp == 0 and node1!=node2 and g.has_node(node1) and g.has_node(node2): # if edge不在graph裡才要做處理
            negative_edges.add((node1,node2))
        else:
            continue
    return negative_edges

# MODELS

In [3]:
def combine_embedding(data,embeddings):
    i=0
    X = []
    for node1,node2 in data:
        X.append(np.concatenate((data[i],embeddings[int(node1)],embeddings[int(node2)])))
   # print(embeddings[str(int(data[0]))])
        i+=1
    return X

In [4]:
all_ap = []
all_auc = []

filename = 'celegans'

for i in range(10):
    ori_df = pd.read_csv('data/'+filename+'.txt', header=None, sep=' ')
    ori_df.columns = ['id_1', 'id_2']

    ori_G = nx.from_pandas_edgelist(ori_df, 'id_1', 'id_2')
    #print(nx.info(ori_G))
    
    negative_edges = get_negative_edges(ori_df, ori_G)
    #print(len(negative_edges))

    train_df, test_df = train_test_split(ori_df, test_size=0.1)
    
    nodes_number = len(ori_G.nodes)
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(train_df.shape[0]):
        graph_np[train_df.iloc[i, 0], train_df.iloc[i, 1]] = 1
        graph_np[train_df.iloc[i, 1], train_df.iloc[i, 0]] = 1

    train_G = nx.from_numpy_matrix(graph_np)
    #print(nx.info(train_G))
    
    df_neg = pd.DataFrame(list(negative_edges), columns=['id_1', 'id_2'])
    
    train_df = train_df.values
    test_pos_df = test_df.values
    df_neg = df_neg.values
    
    model = LINE(train_G, embedding_size=12, order='all')
    model.train(batch_size=128, epochs=50, verbose=2)
    
    LINE_embeddings = model.get_embeddings()
    
    train_df = combine_embedding(train_df, LINE_embeddings)
    test_pos_df = combine_embedding(test_pos_df, LINE_embeddings)
    df_neg = combine_embedding(df_neg, LINE_embeddings)
    
    X_train_neg, X_test_neg = train_test_split(df_neg, test_size=0.1)
    
    y_train_pos = np.ones(len(train_df))
    y_train_neg = np.zeros(len(X_train_neg))
    y_test_pos = np.ones(len(test_pos_df))
    y_test_neg = np.zeros(len(X_test_neg))
    
    X_train = np.concatenate((train_df, X_train_neg))
    y_train = np.concatenate((y_train_pos, y_train_neg))
    X_test = np.concatenate((test_pos_df, X_test_neg))
    y_test = np.concatenate((y_test_pos, y_test_neg))
    
    clf = RandomForestClassifier(n_estimators=400)
    clf.fit(X_train, y_train)
    
    predict_Y = clf.predict(X_test)
    
    ap = average_precision_score(y_test, predict_Y)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(y_test, predict_Y, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    all_ap.append(ap)
    all_auc.append(auc)

print("####################### SUM UP ############################")
print("AP MEAN : ", np.array(all_ap).mean())
print("AP STD : ", np.array(all_ap).std())
print("AUC MEAN : ", np.array(all_auc).mean())
print("AUC STD : ", np.array(all_auc).std())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/50
91/91 - 0s - loss: 1.3861 - first_order_loss: 0.6932 - second_order_loss: 0.6930
Epoch 2/50
91/91 - 0s - loss: 1.3823 - first_order_loss: 0.6931 - second_order_loss: 0.6893
Epoch 3/50
91/91 - 0s - loss: 1.3607 - first_order_loss: 0.6929 - second_order_loss: 0.6678
Epoch 4/50
91/91 - 0s - loss: 1.3127 - first_order_loss: 0.6928 - second_order_loss: 0.6198
Epoch 5/50
91/91 - 0s - loss: 1.2450 - first_order_loss: 0.6926 - second_order_loss: 0.5525
Epoch 6/50
91/91 - 0s - loss: 1.1824 - first_order_loss: 0.6924 - second_order_loss: 0.4901
Epoch 7/50
91/91 - 0s - loss: 1.1502 - first_order_loss: 0.6919 - second_order_loss: 0.4583
Epoch 8/50
91/91 - 0s - loss: 1.1088 - first_order_loss: 0.6913 - second_order_loss:

Epoch 26/50
92/92 - 0s - loss: 1.0176 - first_order_loss: 0.6722 - second_order_loss: 0.3454
Epoch 27/50
92/92 - 0s - loss: 1.0180 - first_order_loss: 0.6726 - second_order_loss: 0.3454
Epoch 28/50
92/92 - 0s - loss: 1.0234 - first_order_loss: 0.6695 - second_order_loss: 0.3539
Epoch 29/50
92/92 - 0s - loss: 1.0084 - first_order_loss: 0.6687 - second_order_loss: 0.3397
Epoch 30/50
92/92 - 0s - loss: 1.0204 - first_order_loss: 0.6711 - second_order_loss: 0.3493
Epoch 31/50
92/92 - 0s - loss: 1.0167 - first_order_loss: 0.6657 - second_order_loss: 0.3509
Epoch 32/50
92/92 - 0s - loss: 1.0124 - first_order_loss: 0.6672 - second_order_loss: 0.3451
Epoch 33/50
92/92 - 0s - loss: 1.0087 - first_order_loss: 0.6695 - second_order_loss: 0.3392
Epoch 34/50
92/92 - 0s - loss: 1.0180 - first_order_loss: 0.6643 - second_order_loss: 0.3537
Epoch 35/50
92/92 - 0s - loss: 1.0098 - first_order_loss: 0.6690 - second_order_loss: 0.3409
Epoch 36/50
92/92 - 0s - loss: 1.0030 - first_order_loss: 0.6646 - sec

92/92 - 0s - loss: 1.0592 - first_order_loss: 0.6858 - second_order_loss: 0.3734
Epoch 14/50
92/92 - 0s - loss: 1.0491 - first_order_loss: 0.6848 - second_order_loss: 0.3643
Epoch 15/50
92/92 - 0s - loss: 1.0433 - first_order_loss: 0.6832 - second_order_loss: 0.3601
Epoch 16/50
92/92 - 0s - loss: 1.0496 - first_order_loss: 0.6818 - second_order_loss: 0.3678
Epoch 17/50
92/92 - 0s - loss: 1.0413 - first_order_loss: 0.6824 - second_order_loss: 0.3590
Epoch 18/50
92/92 - 0s - loss: 1.0317 - first_order_loss: 0.6810 - second_order_loss: 0.3507
Epoch 19/50
92/92 - 0s - loss: 1.0380 - first_order_loss: 0.6785 - second_order_loss: 0.3594
Epoch 20/50
92/92 - 0s - loss: 1.0324 - first_order_loss: 0.6795 - second_order_loss: 0.3530
Epoch 21/50
92/92 - 0s - loss: 1.0250 - first_order_loss: 0.6772 - second_order_loss: 0.3478
Epoch 22/50
92/92 - 0s - loss: 1.0380 - first_order_loss: 0.6737 - second_order_loss: 0.3643
Epoch 23/50
92/92 - 0s - loss: 1.0237 - first_order_loss: 0.6760 - second_order_lo

AP：  0.7434248977206313
AUC SCORE:  0.8114406779661016
Epoch 1/50
92/92 - 0s - loss: 1.3862 - first_order_loss: 0.6932 - second_order_loss: 0.6931
Epoch 2/50
92/92 - 0s - loss: 1.3832 - first_order_loss: 0.6931 - second_order_loss: 0.6901
Epoch 3/50
92/92 - 0s - loss: 1.3615 - first_order_loss: 0.6930 - second_order_loss: 0.6685
Epoch 4/50
92/92 - 0s - loss: 1.3077 - first_order_loss: 0.6929 - second_order_loss: 0.6149
Epoch 5/50
92/92 - 0s - loss: 1.2379 - first_order_loss: 0.6927 - second_order_loss: 0.5452
Epoch 6/50
92/92 - 0s - loss: 1.1775 - first_order_loss: 0.6925 - second_order_loss: 0.4850
Epoch 7/50
92/92 - 0s - loss: 1.1413 - first_order_loss: 0.6919 - second_order_loss: 0.4493
Epoch 8/50
92/92 - 0s - loss: 1.1068 - first_order_loss: 0.6915 - second_order_loss: 0.4153
Epoch 9/50
92/92 - 0s - loss: 1.0813 - first_order_loss: 0.6906 - second_order_loss: 0.3907
Epoch 10/50
92/92 - 0s - loss: 1.0817 - first_order_loss: 0.6899 - second_order_loss: 0.3917
Epoch 11/50
92/92 - 0s -

92/92 - 0s - loss: 1.0036 - first_order_loss: 0.6652 - second_order_loss: 0.3384
Epoch 39/50
92/92 - 0s - loss: 0.9974 - first_order_loss: 0.6656 - second_order_loss: 0.3317
Epoch 40/50
92/92 - 0s - loss: 1.0107 - first_order_loss: 0.6641 - second_order_loss: 0.3467
Epoch 41/50
92/92 - 0s - loss: 0.9994 - first_order_loss: 0.6661 - second_order_loss: 0.3333
Epoch 42/50
92/92 - 0s - loss: 0.9963 - first_order_loss: 0.6680 - second_order_loss: 0.3283
Epoch 43/50
92/92 - 0s - loss: 1.0053 - first_order_loss: 0.6620 - second_order_loss: 0.3433
Epoch 44/50
92/92 - 0s - loss: 0.9919 - first_order_loss: 0.6639 - second_order_loss: 0.3280
Epoch 45/50
92/92 - 0s - loss: 1.0013 - first_order_loss: 0.6667 - second_order_loss: 0.3346
Epoch 46/50
92/92 - 0s - loss: 1.0064 - first_order_loss: 0.6644 - second_order_loss: 0.3420
Epoch 47/50
92/92 - 0s - loss: 0.9861 - first_order_loss: 0.6627 - second_order_loss: 0.3234
Epoch 48/50
92/92 - 0s - loss: 0.9897 - first_order_loss: 0.6644 - second_order_lo

Epoch 26/50
92/92 - 0s - loss: 1.0149 - first_order_loss: 0.6723 - second_order_loss: 0.3425
Epoch 27/50
92/92 - 0s - loss: 1.0161 - first_order_loss: 0.6734 - second_order_loss: 0.3427
Epoch 28/50
92/92 - 0s - loss: 1.0268 - first_order_loss: 0.6702 - second_order_loss: 0.3566
Epoch 29/50
92/92 - 0s - loss: 1.0137 - first_order_loss: 0.6718 - second_order_loss: 0.3419
Epoch 30/50
92/92 - 0s - loss: 1.0082 - first_order_loss: 0.6702 - second_order_loss: 0.3380
Epoch 31/50
92/92 - 0s - loss: 1.0107 - first_order_loss: 0.6665 - second_order_loss: 0.3441
Epoch 32/50
92/92 - 0s - loss: 1.0084 - first_order_loss: 0.6699 - second_order_loss: 0.3385
Epoch 33/50
92/92 - 0s - loss: 1.0094 - first_order_loss: 0.6698 - second_order_loss: 0.3396
Epoch 34/50
92/92 - 0s - loss: 1.0174 - first_order_loss: 0.6662 - second_order_loss: 0.3512
Epoch 35/50
92/92 - 0s - loss: 1.0016 - first_order_loss: 0.6661 - second_order_loss: 0.3356
Epoch 36/50
92/92 - 0s - loss: 0.9972 - first_order_loss: 0.6657 - sec